In [1]:
using LCMGL
using GeometryTypes
import SoftRobots
using Meshing

INFO: Recompiling stale cache file /home/rdeits/.julia/lib/v0.4/Meshing.ji for module Meshing.
INFO: Recompiling stale cache file /home/rdeits/.julia/lib/v0.4/SoftRobots.ji for module SoftRobots.


In [13]:
module Drawing

using LCMGL
using GeometryTypes
using Meshing

function draw(lcmgl::LCMGLClient, mesh; face_color=[.7, .7, .2, .4])
    color(lcmgl, face_color...)
    begin_mode(lcmgl, LCMGL.TRIANGLES)
    verts = vertices(mesh)
    for face in faces(mesh)
        for point in verts[face][end:-1:1]
            vertex(lcmgl, point[1], point[2], point[3])
        end
    end
    end_mode(lcmgl)
    color(lcmgl, 0., 0., 0., 1.0)
    begin_mode(lcmgl, LCMGL.LINES)
    for face in faces(mesh)
        for i = 1:length(verts[face])
            vertex(lcmgl, verts[face][i]...)
            j = i + 1
            if i == length(verts[face])
                j = 1
            end
            vertex(lcmgl, verts[face][j]...)
        end
    end
    normals = decompose(Normal{3, Float64}, mesh)
    for i = 1:length(verts)
      vertex(lcmgl, verts[i]...)
      vertex(lcmgl, (verts[i] + Point(normals[i]) * 0.05)...)
    end
    end_mode(lcmgl)
    # switch_buffer(lcmgl)
end


function draw(lcmgl::LCMGLClient, robot, state)
    body_mesh = SoftRobots.HomogenousMesh(state.positions, robot.faces)
  draw(lcmgl, body_mesh)
  switch_buffer(lcmgl)

  LCMGLClient("barrier") do lcmgl
    center = mean(state.positions)
    width = 1.5*(maximum(state.positions) - minimum(state.positions))

        boundary_mesh = SoftRobots.HomogenousMesh(
            SignedDistanceField(x -> SoftRobots.evaluate(state.barrier, x), 
        HyperRectangle(Vec(center), Vec(width)), 0.01), 0.0)
        draw(lcmgl, boundary_mesh, face_color=[.8, .1, .1, .3])
    switch_buffer(lcmgl)
  end
end

end
import Drawing

In [14]:

# robot, state = SoftRobots.tetrahedron()
# points = state.positions
points = Point{3, Float64}[[1;0;0], [0;1;0], [-1;0;0], [0;-1;0], [0;0;1], [0;0;-1]]
for i = 1:length(points)
    points[i] = 0.25 * points[i] + Point(0.5,0.5,0.5)
end
faces = SoftRobots.convex_hull(points)

8-element Array{GeometryTypes.Face{3,Int64,0},1}:
 Face(5,2,3)
 Face(1,2,5)
 Face(3,2,6)
 Face(6,2,1)
 Face(1,5,4)
 Face(4,5,3)
 Face(3,6,4)
 Face(4,6,1)

In [15]:
body_mesh = HomogenousMesh(points, faces)

HomogenousMesh(
    vertices: 6xFixedSizeArrays.Point{3,Float64},     faces: 8xGeometryTypes.Face{3,Int64,0}, )


In [16]:
body_normals = decompose(Normal{3, Float64}, body_mesh)

6-element Array{GeometryTypes.Normal{3,Float64},1}:
 Normal(1.0,0.0,0.0) 
 Normal(0.0,1.0,0.0) 
 Normal(-1.0,0.0,0.0)
 Normal(0.0,-1.0,0.0)
 Normal(0.0,0.0,1.0) 
 Normal(0.0,0.0,-1.0)

In [17]:
LCMGLClient("body") do lcmgl
    Drawing.draw(lcmgl, body_mesh)
    switch_buffer(lcmgl)
end

In [18]:
barrier = SoftRobots.HermiteRadialField(points, body_normals)

SpatialFields.HermiteRadialField{3,Float64}([1.0448154998549626,1.0448154998549695,1.0448154998549641,1.0448154998549692,1.0448154998549621,1.0448154998549641],6-elementArray{FixedSizeArrays.Point{3,Float64},1}:
 Point(0.17413591664249384,-2.4568415227160405e-17,8.85932549201422e-17)
 Point(3.509757877716747e-17,0.1741359166424947,-2.0066479623783028e-16)
 Point(-0.17413591664249406,4.3384838383612615e-17,-1.3615918177141855e-16)
 Point(-2.3256740821815874e-17,-0.17413591664249475,-1.3432770785901858e-16)
 Point(-1.2030777509140825e-16,7.563794697006869e-17,0.1741359166424939)
 Point(9.547221689661593e-17,9.9896194281274e-18,-0.17413591664249406)
,6-elementArray{FixedSizeArrays.Point{3,Float64},1}:
 Point(0.75,0.5,0.5)
 Point(0.5,0.75,0.5)
 Point(0.25,0.5,0.5)
 Point(0.5,0.25,0.5)
 Point(0.5,0.5,0.75)
 Point(0.5,0.5,0.25)
,SpatialFields.PhiXCubed())

In [66]:
center = Vec(mean(points))
widths = Vec(1.25*(maximum(points) - minimum(points)))
lb = Vec(center - 0.5*widths)
ub = Vec(center + 0.5*widths)
bounds = HyperRectangle(lb, widths)
barrier_field = SignedDistanceField(x -> SoftRobots.evaluate(barrier, x), bounds, maximum(widths)/10);

In [67]:
lb

FixedSizeArrays.Vec{3,Float64}((0.1875,0.1875,0.1875))

In [68]:
ub

FixedSizeArrays.Vec{3,Float64}((0.8125,0.8125,0.8125))

In [69]:
maximum(bounds)

FixedSizeArrays.Vec{3,Float64}((0.8125,0.8125,0.8125))

In [84]:
transform(barrier_inds) = Point((Vec{3, Float64}(barrier_inds) - 1) ./ (Vec(size(barrier_field)) - 1) .* (ub - lb) + lb)

transform (generic function with 1 method)

In [86]:
inds = [7,7,7]
@show barrier_field[inds...]
# @show transform(inds)
@show SoftRobots.evaluate(barrier, transform(inds))

barrier_field[inds...] = -0.08468676308148321

-0.08468676308148321


SoftRobots.evaluate(barrier,transform(inds)) = -0.08468676308148321


In [82]:
barrier_mesh = HomogenousMesh(barrier_field, 0.0)
barrier_points = Point{3,Float64}[Vec(v-1) ./ (Vec(size(barrier_field))-1) .* (ub - lb) + lb for v in vertices(barrier_mesh)]
barrier_mesh = HomogenousMesh(barrier_points, barrier_mesh.faces)

HomogenousMesh(
    vertices: 854xFixedSizeArrays.Point{3,Float64},     faces: 1704xGeometryTypes.Face{3,Int64,0}, )


In [83]:
LCMGLClient("barrier") do lcmgl
    Drawing.draw(lcmgl, barrier_mesh; face_color=[.8,.2,.2,.3])
    switch_buffer(lcmgl)
end